In [1]:
import pandas as pd

# Load the uploaded dataset to inspect the contents
file_path = 'collisions.csv'
data = pd.read_csv(file_path)


In [2]:
# Standardize the road names in the 'Road Location' column to lowercase
data['Road Location'] = data['Road Location'].str.lower()

# Group by the standardized road names
grouped_data = data.groupby('Road Location').size().reset_index(name='Count')
# Consolidate all road names starting with "highway" into one category "highway"
data['Road Location'] = data['Road Location'].str.strip()  # Remove extra spaces
data['Road Location'] = data['Road Location'].apply(
    lambda x: 'highway' if isinstance(x, str) and x.startswith('highway') else x
)

# Consolidate all road names starting with "hwy " into one category "highway"
data['Road Location'] = data['Road Location'].apply(
    lambda x: 'highway' if isinstance(x, str) and x.startswith('hwy ') else x
)

# Re-group and re-sort the data after consolidation
final_grouped_data = data.groupby('Road Location').size().reset_index(name='Count')
final_sorted_data = final_grouped_data.sort_values(by='Count', ascending=False)
final_sorted_data.to_csv("temp.csv")


In [3]:
import plotly.express as px

# Filter data for Bedford Highway and others
bedford_hwy_data = data[data['Road Location'] == 'bedford hwy']
other_data = data[data['Road Location'] != 'bedford hwy']

# Create the map
fig = px.scatter_mapbox(
    data_frame=data,
    lat="Latitude WGS84",
    lon="Longitude WGS84",
    color=data['Road Location'].apply(lambda x: 'Bedford Hwy' if x == 'bedford hwy' else 'Others'),
    title="Road Collisions Highlighting Bedford Highway (Nova Scotia)",
    mapbox_style="carto-positron",
    zoom=10,
    height=600
)

# Highlight Bedford Highway in red
fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.update_layout(legend_title_text='Road Category')

# Show the map
fig.show()


In [5]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load or define your dataset here
# Assuming `data` contains the full dataset
# data = pd.read_csv('path_to_your_file.csv')  # Load your CSV if needed

# Consolidate road names like "hwy"
data['Road Location'] = data['Road Location'].apply(
    lambda x: 'highway' if isinstance(x, str) and x.startswith('hwy ') else x
)

# Get the top 10 roads by count
final_grouped_data = data.groupby('Road Location').size().reset_index(name='Count')
final_sorted_data = final_grouped_data.sort_values(by='Count', ascending=False)
top_10_roads = final_sorted_data.head(10)['Road Location'].tolist()

# Filter the data for the top 10 roads
top_10_data = data[data['Road Location'].isin(top_10_roads)].dropna(subset=['Latitude WGS84', 'Longitude WGS84'])

# Create a Dash app
app = Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Interactive Top 10 Roads Collisions Map"),
    
    # Checklist for road selection
    dcc.Checklist(
        id='road-selector',
        options=[{'label': road, 'value': road} for road in top_10_roads],
        value=top_10_roads,  # Select all roads by default
        inline=True
    ),
    
    # Label for collision count
    html.Div(id='collision-count', style={'marginTop': '20px', 'fontSize': '18px'}),
    
    # Map display
    dcc.Graph(id='collision-map')
])

# Callback to update the map and the collision count
@app.callback(
    [Output('collision-map', 'figure'),
     Output('collision-count', 'children')],
    Input('road-selector', 'value')
)
def update_map_and_count(selected_roads):
    # Filter the data based on selected roads
    filtered_data = top_10_data[top_10_data['Road Location'].isin(selected_roads)]
    
    # Generate the figure
    fig = px.scatter_mapbox(
        filtered_data,
        lat="Latitude WGS84",
        lon="Longitude WGS84",
        hover_name="Road Location",
        title="Collisions on Selected Roads",
        zoom=10,
        color="Road Location",
    )
    
    # Update layout for a black-and-white map
    fig.update_layout(mapbox_style="carto-positron")
    
    # Calculate collision count
    collision_count = len(filtered_data)
    count_label = f"Total Collisions: {collision_count}" if collision_count else "No collisions for the selected road(s)."
    
    return fig, count_label

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8097)
